In [67]:
import pandas as pd
import numpy as np

np.random.seed(1)

In [68]:
od_data_folder = '/Users/ashrafzaman/matsim_pipelines/DataDonwloaded/EOD_from_Louiselle/EOD'
od_data = pd.read_csv(od_data_folder+"/mtl18pv2c.csv",delimiter=";",low_memory=False)
pp = od_data[['nolog','noper','tper','sexe','age','grpage','incap','percond','abon_ap','abon_vls','passetc','occper','tele_trav','lieuocc','xmtmocc','ymtmocc','rmrocc','arrocc','sdrocc','srocc','smocc','cpocc','l_id_local','mobil','facper','facper18']]
hh = od_data[['nolog','tlog','nbper','nbveh','revenu','langue','nmodif_ass','incaplogi','xmtmlog','ymtmlog','r6log','r8log','rmrlog','arrlog','sdrlog','srlog','sdomi65','smlog','cplog','agr_domi','jour','sem','facmen','facmen18']]
dep = od_data[['nolog','noper','nodep','no_rep','hredep','ghredep','motif','motif_grp','date_dpl','jour_dpl','mode1','mode2','mode3','mode4','mode5','mode6','mode7','mode8','seq_modes','pers_auto','auto_ap','velo_ls','station_p','xmtmori','ymtmori','xlonori','ylatori','xmtmdes','ymtmdes','xlondes','ylatdes','ipere','facper18']].dropna()
pp['id_personne'] = od_data.nolog.astype(str)+"_"+od_data.noper.astype(str)
dep['id_personne'] = od_data.nolog.astype(str)+"_"+od_data.noper.astype(str)
dep['id_deplacement'] = od_data.nolog.astype(str)+"_"+od_data.noper.astype(str)+"_"+od_data.nodep.astype(str)
dep['hredep_dec'] = ((dep.hredep).astype(int)/100).astype(int)+dep.hredep.astype(int)%100/60

/var/folders/v6/djhtsx5d3198mkwrr88jb6tc0000gn/T/ipykernel_16645/1792501799.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp['id_personne'] = od_data.nolog.astype(str)+"_"+od_data.noper.astype(str)


In [69]:
def infer_income(hh_size, hh_cars, hh_workers, hh_ct, hh_income):
    if hh_income > 8:
        candidates = hh[(hh.CTUID == hh_ct) & (hh.hh_car == hh_cars) & (hh.hh_size == hh_size)
                        & (hh.hh_workers == hh_workers) & (hh.hh_income < 9)]
        if len(candidates):
            return np.random.choice(candidates.hh_income.values)
        else:
            return np.random.choice(range(1,9)) # in case no household resembles the one with the missing data, return random income class
    else:
        return hh_income

## Filter data from the CMA of Montreal

In [70]:
hh['cma'] = [str(x)[:3] for x in hh.srlog]
hh = hh[hh['cma'] == '462'].copy()

/var/folders/v6/djhtsx5d3198mkwrr88jb6tc0000gn/T/ipykernel_16645/4092956540.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hh['cma'] = [str(x)[:3] for x in hh.srlog]


In [71]:
hh.drop_duplicates()
hh.dropna()


,nolog,tlog,nbper,nbveh,revenu,langue,nmodif_ass,incaplogi,xmtmlog,ymtmlog,...,srlog,sdomi65,smlog,cplog,agr_domi,jour,sem,facmen,facmen18,cma
0,101006,1.0,2,0,1,1,0,1,295187,5045113,...,4620239.00,7,107,H2R2R2,A,2,36,"21,41","21,89",462
7,101037,1.0,1,1,1,2,0,2,292481,5035527,...,4620098.00,25,104,H4W3K7,A,2,36,"29,32","30,3",462
9,101049,1.0,1,0,1,1,0,2,296257,5043924,...,4620218.00,7,107,H2S2T1,A,3,36,"28,38","29,73",462
17,101057,1.0,3,3,3,1,0,2,297139,5054579,...,4620290.09,14,114,H1E2S8,A,1,39,"48,8","51,18",462
23,101060,1.0,1,0,9,1,0,2,293888,5043769,...,4620224.00,7,107,H3N2Z2,A,1,37,"28,86","30,23",462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393786,810241,1.0,2,0,3,1,0,2,300272,5045373,...,4620187.01,10,110,H1W3E2,A,5,51,"28,28","30,03",462
393805,810248,1.0,3,3,9,2,0,2,298671,5051891,...,4620594.02,16,116,H1J1V6,A,5,51,"26,62","31,51",462
393813,810253,1.0,2,1,9,1,0,2,300914,5049355,...,4620191.00,12,112,H1M1M4,A,5,51,"18,75","17,82",462
393815,810256,1.0,2,1,3,1,0,2,282900,5040812,...,4620511.01,36,136,H8Y3T1,A,5,51,"22,24","23,42",462


In [72]:
pp = pp[pp.nolog.isin(hh.nolog.unique())]

### Check

In [73]:
assert len(pp.nolog.unique()) == len(hh.nolog.unique())  # Check that every person is assigned to household

## Households

### HH types (kids or No)

In [74]:
hh_with_kids = pp[pp['age'] < 19]['nolog'].unique()  # households with at least one child under 15 of age

In [75]:
hh_kids = hh.loc[hh['nolog'].isin(hh_with_kids)] # Do not include households that are solely formed by someone under 19y/o
hh_with_kids = hh_kids[hh_kids.nbper > 1].nolog.values

In [76]:
hh['hh_type'] = 2
hh.loc[hh['nolog'].isin(hh_with_kids), 'hh_type'] = 1

In [77]:
assert len(hh[hh['hh_type'] == 1]) == len(hh_with_kids)

### New IDs 

In [78]:
hh_id_dict = dict(zip(pd.np.sort(list(hh.nolog)), range(len(hh))))

/var/folders/v6/djhtsx5d3198mkwrr88jb6tc0000gn/T/ipykernel_16645/3729937750.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  hh_id_dict = dict(zip(pd.np.sort(list(hh.nolog)), range(len(hh))))


In [79]:
hh['hid'] = [hh_id_dict[x] for x in hh.nolog]

## New ID for geos

In [80]:
hh.rename(columns={'srlog': 'CTUID'}, inplace=True) # depending on the EOD (2013 or 2016), census tracts are not the same
hh['CTUID'] *= 1000

In [81]:
mapping_da_ct = gpd.read_csv("V_1/matching_da_ct.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'V_1/matching_da_ct.csv'

In [314]:
park_cts = hh[~hh.CTUID.isin(mapping_da_ct.CTUID)].CTUID.unique()

The above unmatched census tracts are parks and gardens where no census data are available. The HTS inclued these CT due to fuzzy assignment ou CT to HH. We address this problem by assining a random CT near the original one (that indicated in the HTS).

In [315]:
hh.loc[hh.CTUID.isin(park_cts), 'CTUID'] += 1000  # 1000 because I have multiplied by 1000 before performing the join

In [316]:
hh = hh.merge(mapping_da_ct[["CTUID", 'new_ct']].drop_duplicates(), on='CTUID')

In [317]:
hh.rename(columns={"new_ct":"sample_geo", 'nbper': 'hh_size', 'nbveh': 'hh_car'}, inplace=True)

### HH size 

In [318]:
hh.loc[hh.hh_size > 4, "hh_size"] = 5

### HH car

In [319]:
hh.loc[hh.hh_car > 4, 'hh_car'] = 4
hh.loc[hh.hh_car == 0, 'hh_car'] = 5  # HHs without car

### HH income

In [320]:
hh.rename(columns={"revenu":"hh_income"}, inplace=True)

### 16% of HHs do not indicate their income. We need to infer this from the distribution of incomes of neighbouring HHs. We use HH size, Number of cars and Number of workers and Census tracts

In [321]:
nb_workers = pp[pp.occper < 3].groupby("nolog", as_index=False).count()[["nolog", "noper"]].rename(columns={"noper":"hh_workers"})

In [322]:
hh = hh.merge(nb_workers, how="left")

In [323]:
hh["hh_income"] = np.vectorize(infer_income)(hh.hh_size, hh.hh_car, hh.hh_workers, hh.CTUID, hh.hh_income)

In [324]:
hh.loc[hh.hh_income == 5, "hh_income"] = 4
hh.loc[hh.hh_income.isin([6, 7, 8]), "hh_income"] = 5

## Persons

### Age

+ 1 : 0 à 4 ans
+ 2 : 5 à 9 ans
+ 3 : 10 à 14 ans
+ 4 : 15 à 19 ans
+ 5 : 20 à 24 ans
+ 6 : 25 à 29 ans
+ 7 : 30 à 34 ans
+ 8 : 35 à 39 ans
+ 9 : 40 à 44 ans
+ 10 : 45 à 49 ans
+ 11 : 50 à 54 ans
+ 12 : 55 à 59 ans
+ 13 : 60 à 64 ans
+ 14 : 65 à 69 ans
+ 15 : 70 à 74 ans
+ 16 : 75 ans et +


### Age 

In [327]:
def compute_age_category(age):
    if age >= 80:
        result = 16
    else:
        result = age // 5 + 1
    return result

In [328]:
pp['age'] = pp['age'].apply(compute_age_category)

### Driving license 

In [329]:
pp['p_licence'] = pp['percond']
pp.loc[(pp['p_licence'] == 3) | (pp['p_licence'] == 4), 'p_licence'] = 2  # Don't know or Don't want to answer consider as no Driving licence

In [330]:
pp.loc[pp.p_licence == 1, 'license_sex'] = pp.p_licence * pp.sexe
pp.loc[pp['license_sex'].isna(), 'license_sex'] = 3
pp['license_sex'] = pp['license_sex'].astype(int)

In [331]:
pp.loc[pp.p_licence == 5, 'p_licence'] = 3
pp['dummy_license'] = pp['p_licence'] // 2

In [332]:
pp.loc[pp.dummy_license == 0, 'dummy_license'] = 2
pp.dummy_license -= 1

In [333]:
test = pp.groupby(['nolog'])["dummy_license"].sum()

In [334]:
hh = hh.merge(test, on='nolog')
hh.rename(columns={'dummy_license': 'hh_driving_license'}, inplace=True)
hh.loc[hh.hh_driving_license > 4, 'hh_driving_license'] = 4
hh.loc[hh.hh_driving_license == 0, 'hh_driving_license'] = 5  # Rename HHs without a driving license category to 5

### Occupation 

Not used for the V0

#### Workplace

Not used for the V0

In [335]:
pp

,nolog,noper,tper,sexe,age,grpage,incap,percond,abon_ap,abon_vls,...,srocc,smocc,cpocc,l_id_local,mobil,facper,facper18,p_licence,license_sex,dummy_license
0,101006,1,1.0,2,16,16,2,2,4,2,...,NaN,NaN,NaN,0,2,24.48,24.14,2,3,0
1,101006,2,1.0,1,16,16,1,2,4,2,...,NaN,NaN,NaN,0,1,21.82,23.02,2,3,0
2,101021,1,1.0,1,8,8,4,1,2,2,...,4620063.0,101.0,NaN,3,1,32.70,35.96,1,1,1
3,101021,2,1.0,2,8,8,4,1,2,2,...,4620062.0,101.0,H3B2G2,4,1,23.52,24.71,1,2,1
4,101037,1,1.0,2,16,16,4,1,2,2,...,NaN,NaN,NaN,0,1,25.67,26.26,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168878,810256,1,1.0,1,14,14,2,1,2,2,...,NaN,NaN,NaN,0,1,20.48,21.31,1,1,1
168879,810256,2,1.0,2,14,14,2,1,2,2,...,NaN,NaN,NaN,0,2,20.12,20.65,1,2,1
168880,810258,1,1.0,1,11,11,2,1,2,2,...,4620056.0,101.0,NaN,3,1,18.30,18.51,1,1,1
168881,810258,2,1.0,2,11,11,2,1,2,2,...,4620098.0,104.0,NaN,5,1,21.88,23.78,1,2,1


In [336]:
pp['hid'] = [hh_id_dict[x] for x in pp.nolog]

In [337]:
pp['pid'] = range(len(pp))

In [338]:
pp = pp.merge(hh[['hid', 'sample_geo']], on='hid')

In [339]:
pp

,nolog,noper,tper,sexe,age,grpage,incap,percond,abon_ap,abon_vls,...,l_id_local,mobil,facper,facper18,p_licence,license_sex,dummy_license,hid,pid,sample_geo
0,101006,1,1.0,2,16,16,2,2,4,2,...,0,2,24.48,24.14,2,3,0,0,0,255
1,101006,2,1.0,1,16,16,1,2,4,2,...,0,1,21.82,23.02,2,3,0,0,1,255
2,101021,1,1.0,1,8,8,4,1,2,2,...,3,1,32.70,35.96,1,1,1,1,2,372
3,101021,2,1.0,2,8,8,4,1,2,2,...,4,1,23.52,24.71,1,2,1,1,3,372
4,101037,1,1.0,2,16,16,4,1,2,2,...,0,1,25.67,26.26,1,2,1,2,4,103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153357,810256,1,1.0,1,14,14,2,1,2,2,...,0,1,20.48,21.31,1,1,1,66404,153394,439
153358,810256,2,1.0,2,14,14,2,1,2,2,...,0,2,20.12,20.65,1,2,1,66404,153395,439
153359,810258,1,1.0,1,11,11,2,1,2,2,...,3,1,18.30,18.51,1,1,1,66405,153396,115
153360,810258,2,1.0,2,11,11,2,1,2,2,...,5,1,21.88,23.78,1,2,1,66405,153397,115


In [340]:
pp.rename(columns={"sexe" : "sex", "CTUID": "sample_geo"}, inplace=True)

In [341]:
pp[pp.sample_geo.isna()]

,nolog,noper,tper,sex,age,grpage,incap,percond,abon_ap,abon_vls,...,l_id_local,mobil,facper,facper18,p_licence,license_sex,dummy_license,hid,pid,sample_geo


In [222]:
pp.columns

Index(['nolog', 'noper', 'tper', 'sex', 'age', 'grpage', 'incap', 'percond',
       'abon_ap', 'abon_vls', 'passetc', 'occper', 'tele_trav', 'lieuocc',
       'xmtmocc', 'ymtmocc', 'rmrocc', 'arrocc', 'sdrocc', 'srocc', 'smocc',
       'cpocc', 'l_id_local', 'mobil', 'facper', 'facper18', 'age_class',
       'p_licence', 'license_sex', 'dummy_license', 'hid', 'pid',
       'sample_geo'],
      dtype='object')

In [342]:
hh[["hid", "sample_geo", "hh_size" ,"hh_type", "hh_car", "hh_driving_license","hh_income", "CTUID"]].astype(int).to_csv("V_2016/household_sample.csv", index=False)

In [343]:
pp[["hid", "pid", "sample_geo", "sex", "age", "dummy_license", "license_sex"]].astype(int).to_csv("V_2016/person_sample.csv", index=False)